In [14]:
import numpy as np
from configs import config
from modules import channel, node, system as sys
from IPython.display import clear_output
import matplotlib.pyplot as plt
import cvxpy as cp
from time import time

In [15]:
# Network topology settings
M = 10
N = 100
K = 20


In [16]:
s = sys.System(M=M, N=N, K=K)

# The configuration settings
s.tau_p = 10
s.M0 = 2

In [17]:
# Assign random transmit power for each UE
for UE in s.UE_list:
    # UE.P = config.P_MAX_UE
    UE.P = np.random.uniform(0.0, 0.5)

In [18]:
s.assign_APs()
s.assign_pilot()
s.calculate_U()

In [19]:
start = time()
s.solve_optimization_jointly_W_P()
end = time()
print("Average accumulated power = {} mW. Processing takes {} seconds".format(s.calculate_upper_bound_UE_k_using_analysis(k=0) * 1000, end - start))

-21.294883245646094
2.5675998267897784
2.5648585157283437
2.5725905273592047
2.571872557225534
2.566937646095349
2.566925084579151
2.562829077039021
2.56282832808711
2.559063449322336
2.5590652946765857


KeyboardInterrupt: 

In [20]:
# random beamforming
start = time()
s.assign_pilot(option='random')
end = time()
print("Random beamforming: {} mW. Processing takes {} seconds.". format(s.calculate_upper_bound_UE_k_using_analysis(k=1) * 1000, end - start))

# optimal W
start = time()
x_, z_ = s.solve_W_by_fixing_P()
end = time()
print("Optimal beamforming: {} mW. Processing takes {} seconds". format(s.calculate_upper_bound_UE_k_using_analysis(k=1) * 1000, end - start))

# optimal P
start = time()
s.solve_P_by_fixing_W(x=x_, z=z_)
end = time()
print("Optimal transmit power: {} mW. Processing takes {} seconds". format(s.calculate_upper_bound_UE_k_using_analysis(k=1) * 1000, end - start))

Random beamforming: 2.1699685198441543 mW. Processing takes 0.000347137451171875 seconds.
Optimal beamforming: 2.1699685198441543 mW. Processing takes 0.1679389476776123 seconds
Optimal transmit power: 2.169968573272562 mW. Processing takes 3.013580799102783 seconds


FIX P SOLVE W

In [ ]:
a = config.a_PARAMETER
b = config.b_PARAMETER
c = config.c_PARAMETER
E_max = config.E_MAX

# parameters
D = (s.tau_c - s.tau_p) * E_max / (1 - c) / s.tau_c
F = D * c
p = np.array([s.UE_list[i].P for i in range(s.K)]).reshape(s.K, 1)
vector_f = [
    [s.construct_vector_f_mk(m=m, k=k) if len(s.AP_list[m].UE_index) > 0 else None for k in range(s.K)] for m in range(s.M)
]

# initiate a value of alpha
alpha = np.ones((s.K, 1))

# tolerance and number of trials
tol = 10**-3
num_trial = 0

# previous and current optimal value
pre_optimal_val = -1000
current_optimal_val = 0

# start with random beamforming
s.assign_beamforming(option='random')
eh = s.calculate_average_accumulated_power()
print("With random - Average harvested power: {:.5f}".format(eh))


while(num_trial < 30) and np.abs(current_optimal_val - pre_optimal_val) > tol:
    num_trial += 1
    # assign the current optimal value to previous one
    pre_optimal_val = current_optimal_val

    # Variables
    W = [cp.Variable((len(AP.UE_index), 1)) if len(AP.UE_index) > 0 else None for AP in s.AP_list]
    x = cp.Variable((s.K, 1))
    y = cp.Variable((s.K, 1))
    z = cp.Variable((s.K, 1))
    # Introduce a slack variable t to control the feasibility of the optimization problem
    t = cp.Variable((s.K, 1), nonneg=True)

    # Constraints
    constr = []
    for w in W:
        if w is not None:
            # constraint 21: C2
            constr += [cp.sum(w) <= 1]
            # constraint 21: C4
            constr += [w >= 0]
    # constraint 22: C2
    constr += [z + y >= a * b]
    # constraint 22: C4
    constr += [x - s.tau_p / s.tau_c / D * p >= F / D]
    # constraint 22: C3
    for k in range(s.K):
        sum = 0
        for m in range(s.M):
            if len(s.AP_list[m].UE_index) > 0:
                sum = sum +  a * s.AP_list[m].P * vector_f[m][k] @ W[m]
        constr += [sum >= z[k]]
    # constraint 25: C1
    constr += [cp.exp(y) + 1 <= 2 * alpha - cp.multiply(x, alpha**2) + t]

    # Objective function
    f = cp.sum(x + 2 * alpha - cp.multiply(x, alpha**2) - cp.inv_pos(x) - t)
    objective = cp.Maximize(f)
    problem = cp.Problem(objective, constr)

    # Solve the problem
    try:
        problem.solve(solver = cp.MOSEK, verbose=False)
        current_optimal_result = problem.value
        # print out the accumulated power
        for i in range(s.M):
            if W[i] is not None:
                s.AP_list[i].w = W[i].value
        eh = s.calculate_average_accumulated_power()
        print("Processing optimization - Average harvested power: {:.5f}".format(eh))

        # assign a new alpha = 1 / x
        alpha = 1/x.value
    except Exception as e:
        print(e)
        break

In [ ]:
z.value

In [ ]:
[a * s.calculate_average_received_power_by_analysis(k) for k in range(s.K)]

In [ ]:
# check all the constraints

# for w in W:
#     if w is not None:
#         print(np.sum(w.value) <= 1)
#         print(w.value >= 0)

# print(z.value + y.value - a * b)

# D * x.value - s.tau_p/s.tau_c * p - F

# np.exp(y.value) + 1 - 2 * alpha + x.value * alpha**2

for k in range(s.K):
    diff = a * np.sum(
        [s.AP_list[m].P * np.matmul(vector_f[m][k], s.AP_list[m].w) for m in range(s.M) if len(s.AP_list[m].UE_index) > 0]
    ) - z.value[k]
    
    print(diff)

FIX W SOLVE P

In [ ]:
# https://math.stackexchange.com/questions/3991820/sum-of-linear-fractional-function-in-constraints-set

In [ ]:
x = x.value
z = z.value

In [ ]:
v = np.zeros(shape=(s.M, s.K))
for m in range(s.M):
    for i in range(s.K):
        vector_u_mi = np.array([s.U[m,j,i]/config.NOISE_POWER/a/s.AP_list[m].P for j in range(s.K)])
        v[m, i] = 1/(np.matmul(vector_u_mi, p ) + 1)

In [ ]:
# parameters
W = s.get_beamforming_weight_W()

# variables
p = cp.Variable((s.K, 1), pos=True)
t = cp.Variable((s.K, 1), nonneg=True)
f = cp.Variable((s.M, s.K), nonneg=True)

for i in range(10):
    # constraints
    constr = []
    constr += [p >= 10**-6]
    constr += [p <= 40]
    constr += [D * x - s.tau_p * p >= F]

    for k in range(s.K):
        sum = 0
        for m in range(s.M):
            for i in s.AP_list[m].UE_index:
                vector_u_mki = np.array([s.U[m,j,i] * s.beta[m,k] if j != k else N * s.U[m,k,i] * s.beta[m,k] for j in range(s.K)])/config.NOISE_POWER
                sum += W[m, i] * v[m, i] * (vector_u_mki @ p + s.beta[m,k] / s.N)
        
        constr += [sum + t[k] >= z[k]]

    for m in range(s.M):
        for i in s.AP_list[m].UE_index:
            vector_u_mi = np.array([s.U[m,j,i]/config.NOISE_POWER/a/s.AP_list[m].P for j in range(s.K)])
            constr += [vector_u_mi @ p + 1/a/s.AP_list[m].P <= 1/v[m, i]]

    # objective function
    obj = - cp.sum(p + t)

    objective = cp.Maximize(obj)
    problem = cp.Problem(objective, constr)
    
    # Solve the problem
    try:
        problem.solve(solver = cp.MOSEK, verbose=False)
        print(problem.value)
        # print out the accumulated power
        for i in range(s.K):
            s.UE_list[i].P = p.value[i, 0]
        eh = s.calculate_average_accumulated_power()
        print("Processing optimization - Average harvested power: {:.5f}".format(eh))

        for m in range(s.M):
            for i in range(s.K):
                vector_u_mi = np.array([s.U[m,j,i]/config.NOISE_POWER/a/s.AP_list[m].P for j in range(s.K)])
                v[m, i] = 1/(np.matmul(vector_u_mi, p.value ) + 1)
    except Exception as e:
        print(e)


In [ ]:
10 * np.log10(p.value * 1000)

In [ ]:
s.optimize_W_by_fixing_P()
s.calculate_average_accumulated_power()